In [280]:
%load_ext lab_black

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


In [281]:
from sklearn import datasets

iris = datasets.load_iris()
X = iris.data
y = iris.target

In [282]:
from sklearn.model_selection import train_test_split

X1, X2, y1, y2 = train_test_split(X, y, test_size=0.40, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(
    X1, y1, test_size=0.40, random_state=42
)

In [283]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(
    max_depth=2, random_state=0, warm_start=False, n_estimators=1
)
model.fit(X_train, y_train)
print(model.score(X_test, y_test))
model.fit(X2, y2)
print(model.score(X2, y2))

0.9444444444444444
0.9833333333333333


In [284]:
model.n_estimators += 1
model.fit(X2, y2)
print(model.score(X_test, y_test))

0.9444444444444444
